In [1]:
import torch
import numpy as np

In [2]:
def knn(data, k=5)->torch.Tensor:
    """Construct edge feature for each point
    Args:
      point_cloud: (batch_size, num_points, num_dims)
      k: int number of neighbours

    Returns:
      idx: shape:(batch_size, num_points, nums_neighours,)
    """
    dists_matrix = torch.cdist(data, data)
    #print(dists_matrix.shape)
    _, idx = dists_matrix.topk(k+1, dim=-1, largest=False)  # +1 the point itself is included
    return idx[...,1:] # not include the point itself



def get_edge_feature(point_cloud, idx=None, k=20,device="cpu"):
    """Construct edge feature for each point
    Args:
      point_cloud: (batch_size, num_points, num_dims)
      idx: (batch_size, num_points, neighbours)
      k: int
      device: cpu/cuda

    Returns:
      features: (batch_size, num_dims ,num_points, k)
    """
    point_cloud = point_cloud.to(device)
    batch_size = point_cloud.shape[0]
    num_points = point_cloud.shape[1]

    if(idx==None):
        idx = knn(point_cloud,k=k) # (batch_size, num_points, nums_neighours,)

    idx_base = torch.arange(0, batch_size, device=device).view(-1, 1, 1) * num_points # create the base index for mapping
    idx = idx.to(device=device)
    idx = idx + idx_base #[0...0...0]->[0...100...200]
    idx=idx.view(-1) # flatten it -> tensor([  0,  56,  25,  ..., 225, 222, 271], device='cuda:0') e.g: [K01,K02,K03,K11,K12,K13...] shape = (B*N*K) 
   
    num_dims = point_cloud.shape[2]

    # feature : turn neighbour index in idx to coordinate
    feature = point_cloud.view(batch_size*num_points, -1)[idx, :] # feature : B*N*F -> BN * F -> (B*N*K) * F
    # feature : reshape into (Batch_size * Num_points *Nums_neigbours * Features)
    feature = feature.view(batch_size, num_points, k, num_dims)
    
    # pointcloud : create replicate of the self point up to k for matching feature - size B*N*K(repeated)*F 
    point_cloud = point_cloud.view(batch_size, num_points, 1, num_dims).repeat(1, 1, k, 1) 

    # feature size B*N*K*F -> B*N*K*2F (feature-x || x)
    feature = torch.cat((feature-point_cloud, point_cloud), dim=3)

    # (B * 2F * N * K) for later conv each coordinate(F)
    feature=feature.permute(0,3,1,2).contiguous()

    return feature

# Example usage:
data = torch.rand((3,100, 3))  # 100 points in 20D (batch_size, num_points, num_dims)
#neighbors = knn(data, k=4)
edges= get_edge_feature(data)
print(edges.shape)
print(type(edges))

torch.Size([3, 6, 100, 20])
<class 'torch.Tensor'>


### **Edgeconv**

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EdgeConv(nn.Module):
    def __init__(self, in_channels, out_channels, num_neighbours=20,device="cpu"):
        """Setup EdgeConv
        Args:
        in_channels: int
        out_channels: int
        num_neighbours: int
        """
        super(EdgeConv, self).__init__()
        self.device=device
        self.k= num_neighbours
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=in_channels*2, out_channels=out_channels, kernel_size=1, bias=False,device=self.device),
            nn.BatchNorm2d(out_channels,device=self.device),
            nn.LeakyReLU(negative_slope=0.2)
        )

    def forward(self,x):
        """Setup EdgeConv
        Args:
        x: shape - (batch_size, num_points, num_dims)

        Returns:
        features: (batch_size, num_dims, num_points, num_neigbours)
        """
        x = get_edge_feature(x, k=self.k,device=self.device) #(batch_size, num_points, dim) -> (batch_size, dim*2, num_points ,k)
        x = self.conv(x)
        # for each point pick the largest k (batch_size, 64, num_points, k) -> (batch_size, 64, num_points)
        x = x.max(dim=-1, keepdim=False)[0]
        x = x.permute(0,2,1).contiguous()
        return x
    
# Example usage:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = torch.rand((3,100, 3))  # 100 points in 20D (batch_size, num_points, num_dims)
conv = EdgeConv(3, 64,device=device)
out = conv(data)
print("out.shape=", out.shape)

out.shape= torch.Size([3, 100, 64])


### **DGCNN (Classification)**

In [4]:
class DGCNN(nn.Module):
    def __init__(self, num_neighbours=20,out_channels=40,dropout_rate =0.3,device="cpu"):
        super(DGCNN,self).__init__()
        self.inChannels=[3,64,64,128,256]
        self.edgeConv0 = EdgeConv(in_channels=3,out_channels=64,num_neighbours=num_neighbours,device=device)
        self.edgeConv1 = EdgeConv(in_channels=64,out_channels=64,num_neighbours=num_neighbours,device=device)
        self.edgeConv2 = EdgeConv(in_channels=64,out_channels=128,num_neighbours=num_neighbours,device=device)
        self.edgeConv3 = EdgeConv(in_channels=128,out_channels=256,num_neighbours=num_neighbours,device=device)

        self.edgeConv4 = EdgeConv(in_channels=512,out_channels=1024,num_neighbours=num_neighbours,device=device)

        self.linear1 = nn.Linear(2048, 512, bias=False,device=device)
        self.bn1 = nn.BatchNorm1d(512,device=device)
        self.drop1 = nn.Dropout(dropout_rate)
        self.linear2 = nn.Linear(512, 256, bias=False,device=device)
        self.bn2 = nn.BatchNorm1d(256,device=device)
        self.drop2 = nn.Dropout(dropout_rate)
        self.linear3 = nn.Linear(256,out_channels, bias=False,device=device)


    def forward(self,x):
        x0=self.edgeConv0(x)
        #print("x0:",x0.shape)
        x1=self.edgeConv1(x0)
        x2=self.edgeConv2(x1)
        x3=self.edgeConv3(x2)

        x=torch.cat((x0,x1,x2,x3),dim=2)
        
        x= self.edgeConv4(x) # (batch_size, num_points ,64+64+128+256) -> (batch_size, num_points, emb_dims(1024))
        
        #todo 
        # maxpool and avgpool
        x= x.permute(0,2,1).contiguous()
        maxPoolX = F.adaptive_avg_pool1d(x,1).view(x.shape[0],-1)
        avgPoolX = F.adaptive_avg_pool1d(x,1).view(x.shape[0],-1)
        x=torch.cat((maxPoolX,avgPoolX),1) #(batch_size, 2048)
        
        #mlp[512,256,c(40)]
        x= F.leaky_relu(self.bn1(self.linear1(x)))
        x=self.drop1(x)
        x= F.leaky_relu(self.bn2(self.linear2(x)))
        x=self.drop2(x)
        x= self.linear3(x)
        # output

        return x
    
# Example usage:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = torch.rand((3,100, 3))  # 100 points in 20D (batch_size, num_points, num_dims)
print("data shape: ", data.shape)
print(device)
dgcnn = DGCNN(device=device)
out = dgcnn(data)

print("out.shape=", out.shape)

data shape:  torch.Size([3, 100, 3])
cuda
out.shape= torch.Size([3, 40])


## Dataset Part

In [5]:
#todo DataSet -> use mobilenet 40

import os
import glob
import h5py
import numpy as np
from torch.utils.data import Dataset

def download_modelnet40():
    BASE_DIR = os.getcwd()
    DATA_DIR = os.path.join(BASE_DIR, 'data')
    print(os.path.exists(os.path.join(DATA_DIR, 'modelnet40_ply_hdf5_2048')))
    if not os.path.exists(DATA_DIR):
        os.mkdir(DATA_DIR)
    if not os.path.exists(os.path.join(DATA_DIR, 'modelnet40_ply_hdf5_2048')):
        www = 'https://shapenet.cs.stanford.edu/media/modelnet40_ply_hdf5_2048.zip'
        zipfile = os.path.basename(www)
        os.system('wget --no-verbose --no-check-certificate %s; unzip %s' % (www, zipfile))
        os.system('mv %s %s' % ('modelnet40_ply_hdf5_2048', DATA_DIR))
        os.system('rm %s' % (zipfile))

def load_data_cls(partition):
    download_modelnet40()
    BASE_DIR = os.getcwd()
    DATA_DIR = os.path.join(BASE_DIR, 'data')
    all_data = []
    all_label = []
    for h5_name in glob.glob(os.path.join(DATA_DIR, 'modelnet40_ply_hdf5_2048', '*%s*.h5'%partition)):
        f = h5py.File(h5_name, 'r+')
        data = f['data'][:].astype('float32')
        label = f['label'][:].astype('int64')
        f.close()
        all_data.append(data)
        all_label.append(label)
    all_data = np.concatenate(all_data, axis=0)
    all_label = np.concatenate(all_label, axis=0)
    return all_data, all_label

def translate_pointcloud(pointcloud):
    xyz1 = np.random.uniform(low=2./3., high=3./2., size=[3])
    xyz2 = np.random.uniform(low=-0.2, high=0.2, size=[3])
       
    translated_pointcloud = np.add(np.multiply(pointcloud, xyz1), xyz2).astype('float32')
    return translated_pointcloud

class ModelNet40(Dataset):
    def __init__(self, num_points, partition='train'):
        self.data, self.label = load_data_cls(partition)
        self.num_points = num_points
        self.partition = partition        

    def __getitem__(self, item):
        pointcloud = self.data[item][:self.num_points]
        label = self.label[item]
        if self.partition == 'train':
            pointcloud = translate_pointcloud(pointcloud)
            np.random.shuffle(pointcloud)
        return pointcloud, label

    def __len__(self):
        return self.data.shape[0]
    
# use like this
train = ModelNet40(1024)
test = ModelNet40(1024, 'test')
data, label = train[0]
print(data.shape)
print(label.shape)

True
True
(1024, 3)
(1,)


## Training part

In [6]:
#todo , not finish 
from torch.utils.data import DataLoader
import os
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
import sklearn.metrics as metrics
def cal_loss(pred, gold, smoothing=True):
    ''' Calculate cross entropy loss, apply label smoothing if needed. '''

    gold = gold.contiguous().view(-1)

    if smoothing:
        eps = 0.2
        n_class = pred.size(1)

        one_hot = torch.zeros_like(pred).scatter(1, gold.view(-1, 1), 1)
        one_hot = one_hot * (1 - eps) + (1 - one_hot) * eps / (n_class - 1)
        log_prb = F.log_softmax(pred, dim=1)

        loss = -(one_hot * log_prb).sum(dim=1).mean()
    else:
        loss = F.cross_entropy(pred, gold, reduction='mean')

    return loss
def train(model, epochs, _lr, numPoints,batchSize,device):
    """train
        Args:
        model: classifier
        dataset: shape(batch_size,nums_point,dimemsion)
        optimizerSelect:1 for SGD, 0 for Adam
        epochs:train epochs
        device:
    """
    train_loader = DataLoader(ModelNet40(partition='train', num_points=numPoints), num_workers=4,
                              batch_size=batchSize, shuffle=True, drop_last=True)
    test_loader = DataLoader(ModelNet40(partition='test', num_points=numPoints), num_workers=4,
                             batch_size=batchSize, shuffle=True, drop_last=False)


    #Try to load models
    model = nn.DataParallel(model)
    opt = optim.Adam(model.parameters(), lr=_lr, weight_decay=1e-4)
    scheduler = CosineAnnealingLR(opt, epochs, eta_min=_lr)
    criterion = cal_loss
    best_test_acc = 0
    for epoch in range(epochs):
        scheduler.step()
        ####################
        # Train
        ####################
        train_loss = 0.0
        count = 0.0
        model.train()
        train_pred = []
        train_true = []
        for data, label in train_loader:

            data, label = data.to(device), label.to(device).squeeze()

            batch_size = data.size()[0]
            opt.zero_grad()
            logits = model(data)
            loss = criterion(logits, label)
            loss.backward()
            opt.step()
            preds = logits.max(dim=1)[1]
            count += batch_size
            train_loss += loss.item() * batch_size
            train_true.append(label.cpu().numpy())
            train_pred.append(preds.detach().cpu().numpy())

        train_true = np.concatenate(train_true)
        train_pred = np.concatenate(train_pred)
        outstr = 'Train %d, loss: %.6f, train acc: %.6f, train avg acc: %.6f' % (epoch,train_loss*1.0/count,metrics.accuracy_score
                                                    (train_true, train_pred),metrics.balanced_accuracy_score
                                                    (train_true, train_pred))
    
        print(outstr)

        ####################
        # Test
        ####################
        test_loss = 0.0
        count = 0.0
        model.eval()
        test_pred = []
        test_true = []
        for data, label in test_loader:
            data, label = data.to(device), label.to(device).squeeze()
            batch_size = data.size()[0]
            logits = model(data)
            loss = criterion(logits, label)
            preds = logits.max(dim=1)[1]
            count += batch_size
            test_loss += loss.item() * batch_size
            test_true.append(label.cpu().numpy())
            test_pred.append(preds.detach().cpu().numpy())
            break
        test_true = np.concatenate(test_true)
        test_pred = np.concatenate(test_pred)
        test_acc = metrics.accuracy_score(test_true, test_pred)
        avg_per_class_acc = metrics.balanced_accuracy_score(test_true, test_pred)
        outstr = 'Test %d, loss: %.6f, test acc: %.6f, test avg acc: %.6f' % (epoch,test_loss*1.0/count,test_acc,avg_per_class_acc)
        print(outstr)
        if test_acc >= best_test_acc:
            best_test_acc = test_acc
            BASE_DIR = os.getcwd()
            DATA_DIR = os.path.join(BASE_DIR, 'checkpoints')
            if not os.path.exists(DATA_DIR):
                os.mkdir(DATA_DIR)
                os.mkdir(os.path.join(DATA_DIR, 'models'))
            torch.save(model.state_dict(), 'checkpoints/models/model.t7')    
    
    
    return 0



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=DGCNN(device=device).to(device)
train(model,100,0.001,1024,8,device)

True
True


d:\anaconda3\Lib\site-packages\torch\optim\lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


## Run Code

In [11]:
## Excute DGCNN


e:\Python\DGCNN
